In [1]:
import os
from google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get("google_api_key")

In [8]:
!pip install -q pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 7.2 MB/s eta 0:00:00


In [2]:
!pip install -q langchain-community langchain-google-genai faiss-cpu tiktoken python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 65.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [36]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from pypdf import PdfReader

In [31]:
Pdf_file = "/content/phapluatDN.pdf"
reader = PdfReader(Pdf_file)
pdf_contain = [page.extract_text() for page in reader.pages]

In [33]:
pdf_contain = ".".join(pdf_contain)
pdf_contain

'QUYẾT ĐỊNH\nBan hành Quy định chức năng, nhiệm vụ, quyền hạn và cơ cấu tổ chức\ncủa Trường Cao đẳng Đà Nẵng\nCăn cứ Luật Tổ chức chính quyền địa phương số 72/2025/QH15 ngày 16 \ntháng 6 năm 2025;\nCăn cứ Luật Ban hành văn bản quy phạm pháp luật số 64/2025/QH15 ngày 19 \ntháng 02 năm 2025; Luật sửa đổi, bổ sung một số điều của Luật Ban hành văn bản \nquy phạm pháp luật số 87/2025/QH15 ngày 25 tháng 6 năm 2025;\nCăn cứ Luật Giáo dục nghề nghiệp số 74/2014/QH13 ngày 27 tháng 11 năm \n2014;\nCăn cứ Nghị định số 78/2025/NĐ-CP ngày 01 tháng 4 năm 2025 của Chính \nphủ quy định chi tiết một số điều và biện pháp để tổ chức, hướng dẫn thi hành Luật \nBan hành văn bản quy phạm pháp luật;\nCăn cứ Nghị định số 187/2025/NĐ-CP ngày 01 tháng 7 năm 2025 của Chính \nphủ Sửa đổi, bổ sung một số điều của Nghị định số 78/2025/NĐ-CP ngày 01 tháng \n4 năm 2025 của Chính phủ quy định chi tiết một số điều và biện pháp để tổ chức, \nhướng dẫn thi hành Luật Ban hành văn bản quy phạm pháp luật và Nghị định số \n

In [34]:
# Chunking
splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
chunks = splitter.create_documents([pdf_contain])

In [37]:
#Tạo 1 VectorDB
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector_store = FAISS.from_documents(chunks, embeddings)

In [38]:
#Retrieve 4 vector gần nhất
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs = {"k":4})
retriever.invoke("Nhiệm vụ, quyền hạn")

[Document(id='5ef1ab5f-86d6-4d4b-be0e-5cf65c305576', metadata={}, page_content='Chính phủ;\nl) Tổ chức hoạt động kiểm định  và bảo đảm chất lượng giáo dục nghề nghiệp \ntheo quy định của Chính phủ;\nm) Xây dựng, đầu tư cơ sở vật chất, thiết bị đào tạo theo yêu cầu chuẩn hóa, \nhiện đại hóa. Sử dụng nguồn thu từ hoạt động đào tạo, khoa học, công nghệ và \ndịch vụ để đầu tư xây dựng cơ sở vật chất của Trường, chi cho các hoạt động đào \ntạo, bổ sung nguồn tài chính của Trường và các hoạt động khác theo quy định của \npháp luật;\nn) Huy động, nhận tài trợ, quản lý và sử dụng nguồn huy động, tài trợ theo \nquy định của pháp luật nhằm thực hiện các hoạt động của Trường;\no) Quản lý, sử dụng tài sản công theo quy định của pháp luật về quản lý, sử dụng \ntài sản công; quản lý và sử dụng nguồn tài chính theo quy định của pháp luật;\np) Được Nhà nước giao hoặc cho thuê đất, giao hoặc cho thuê cơ sở vật chất; \nđược tham gia đấu thầu, đặt hàng hoặc giao nhiệm vụ đối với các dịch vụ công sử \ndụn

In [44]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.2)

prompt = PromptTemplate(
    template="""
      You are a helpful assistance.
      Answer ONLY from the provided transcript context.
      If the context is insufficient, just say you don't know.
      {content}
      Question: {question}
    """,
    input_variables=["content", "question"]
)

In [46]:
question = "Trường Cao đẳng Đà Nẵng thực hiện nhiệm vụ theo cái gì?"
retrieved_docs = retriever.invoke(question)
context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
final_promt = prompt.invoke({"content":context_text, "question":question})
answer = llm.invoke(final_promt)
answer.content

'Trường Cao đẳng Đà Nẵng thực hiện nhiệm vụ theo quy định của pháp luật hiện hành và quy định về phân cấp quản lý cán bộ.'